# Azure SQL Database for Avatar Demo

## Setup

You need to have the following settings for your Azure resources defined in the `local.settings.json` file in the __root__ folder to populate the demo content.

In [ ]:
# Install the required libraries
%pip install pyodbc
%pip install pandas
%pip install requests

In [ ]:
import os
import json  
import pandas as pd

import pyodbc
import requests
import inspect

In [ ]:
# Load JSON file
with open('./local.settings.json', 'r') as file:
    data = json.load(file)

# Blob SAS URL for Azure Storage Account
blob_sas_url = data["Values"]["BLOB_SAS_URL"]

# Azure SQL Database
sql_db_server = data["Values"]["SQL_DB_SERVER"]
sql_db_user = data["Values"]["SQL_DB_USER"]
sql_db_password = data["Values"]["SQL_DB_PASSWORD"]
sql_db_name = data["Values"]["SQL_DB_NAME"]

## Create Azure SQL Database
Now let's create a small Azure SQL database with customer, product and order data using the SQL server you deployed in the previous step.

In [ ]:
# Connection Strings
server_connection_string = f"Driver={{ODBC Driver 17 for SQL Server}};Server=tcp:{sql_db_server},1433;Uid={sql_db_user};Pwd={sql_db_password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
database_connection_string = server_connection_string + f"Database={sql_db_name};"

In [ ]:
import random

customers = [
    {"name": "John Doe", "account_id": 1000, "loyalty_points" : random.randint(400, 800)},
    {"name": "Jane Smith", "account_id": 1001, "loyalty_points" : random.randint(400, 800)},
    {"name": "Alice Johnson", "account_id": 1002, "loyalty_points" : random.randint(400, 800)},
    {"name": "Bob Wilson", "account_id": 1003, "loyalty_points" : random.randint(400, 800)},
    {"name": "Charlie Brown", "account_id": 1004, "loyalty_points" : random.randint(400, 800)},
    {"name": "Eve Adams", "account_id": 1005, "loyalty_points" : random.randint(400, 800)},
    {"name": "Frank Castle", "account_id": 1006, "loyalty_points" : random.randint(400, 800)},
    {"name": "Grace Lee", "account_id": 1007, "loyalty_points" : random.randint(400, 800)},
    {"name": "Hannah Montan", "account_id": 1008, "loyalty_points" : random.randint(400, 800)},
    {"name": "Ian Somerhalder", "account_id": 1009, "loyalty_points" : random.randint(400, 800)},
    {"name": "Peter Mick", "account_id": 1010, "loyalty_points" : random.randint(400, 800)},
]

products = [
    {"id": 1000, "name": "Elysian Voyager", "stock": random.randint(0,50)},
    {"id": 1001, "name": "Terra Roamer", "stock": random.randint(0,50)},
    {"id": 1002, "name": "Cardinal Pathfinder", "stock": random.randint(0,50)},
    {"id": 1003, "name": "Slumber Drifter", "stock": random.randint(0,50)},
    {"id": 1004, "name": "Blaze Adventurer", "stock": random.randint(0,50)},
    {"id": 1005, "name": "BiteShield Pro", "stock": random.randint(0,50)},
    {"id": 1006, "name": "Feast Frontier", "stock": random.randint(0,50)},
    {"id": 1007, "name": "Summit Stride", "stock": random.randint(0,50)},
    {"id": 1008, "name": "Rugged Ranger","stock": random.randint(0,50)},
    {"id": 1100, "name": "Match Master", "stock": random.randint(0,50)},
    {"id": 1101, "name": "Court Queen", "stock": random.randint(0,50)},
    {"id": 1102, "name": "Junior Ace", "stock": random.randint(0,50)},
    {"id": 1103, "name": "ServeMaster Pro", "stock": random.randint(0,50)},
    {"id": 1104, "name": "Court Commander", "stock": random.randint(0,50)},
    {"id": 1105, "name": "StringMaster Elite", "stock": random.randint(0,50)},
    {"id": 1106, "name": "Court Conqueror", "stock": random.randint(0,50)},
    {"id": 1107, "name": "AceMaster 3000", "stock": random.randint(0,50)},
    {"id": 1108, "name": "Ace Attire", "stock": random.randint(0,50)},
    {"id": 1109, "name": "Serve & Style", "stock": random.randint(0,50)},
]
orders = [
    {"order_id": 1000, "product_id": 1001, "days_to_delivery": random.randint(3,15), "account_id": 1000},
    {"order_id": 1001, "product_id": 1001, "days_to_delivery": random.randint(3,15), "account_id": 1001},
    {"order_id": 1002, "product_id": 1002, "days_to_delivery": random.randint(3,15), "account_id": 1002},
    {"order_id": 1003, "product_id": 1003, "days_to_delivery": random.randint(3,15), "account_id": 1003},
    {"order_id": 1004, "product_id": 1004, "days_to_delivery": random.randint(3,15), "account_id": 1004},
    {"order_id": 1005, "product_id": 1005, "days_to_delivery": random.randint(3,15), "account_id": 1005},
    {"order_id": 1006, "product_id": 1006, "days_to_delivery": random.randint(3,15), "account_id": 1006},
    {"order_id": 1007, "product_id": 1007, "days_to_delivery": random.randint(3,15), "account_id": 1007},
    {"order_id": 1008, "product_id": 1008, "days_to_delivery": random.randint(3,15), "account_id": 1008},
    {"order_id": 1010, "product_id": 1000, "days_to_delivery": random.randint(3,15), "account_id": 1009},
    {"order_id": 1012, "product_id": 1101, "days_to_delivery": random.randint(3,15), "account_id": 1001},
    {"order_id": 1013, "product_id": 1102, "days_to_delivery": random.randint(3,15), "account_id": 1002},
    {"order_id": 1014, "product_id": 1103, "days_to_delivery": random.randint(3,15), "account_id": 1003},
    {"order_id": 1015, "product_id": 1104, "days_to_delivery": random.randint(3,15), "account_id": 1004},
    {"order_id": 1016, "product_id": 1105, "days_to_delivery": random.randint(3,15), "account_id": 1005},
    {"order_id": 1017, "product_id": 1106, "days_to_delivery": random.randint(3,15), "account_id": 1006},
    {"order_id": 1018, "product_id": 1107, "days_to_delivery": random.randint(3,15), "account_id": 1007},
    {"order_id": 1019, "product_id": 1108, "days_to_delivery": random.randint(3,15), "account_id": 1008},
]

In [ ]:
# Test connection to the SQL Server

try:
    # Try to establish a connection
    conn = pyodbc.connect(server_connection_string)
    
    # If connection is successful, print a message and close the connection
    print("Connection to the server/database was successful!")
    conn.close()
    
except pyodbc.Error as ex:
    # Catch any connection errors and print them
    sqlstate = ex.args[0] if len(ex.args) > 0 else None
    message = ex.args[1] if len(ex.args) > 1 else None
    print(f"Failed to connect to the server/database. SQLSTATE: {sqlstate}, Message: {message}")

In [ ]:
# SET TO TRUE ONLY TO REBUILD DATABASE BASED ON ABOVE SAMPLE DATA
rebuild_database = True

if rebuild_database:

    # Connect to the server without specifying a database
    server_conn = pyodbc.connect(server_connection_string, autocommit=True)
    server_cursor = server_conn.cursor()

    # Drop the database if it exists
    server_cursor.execute(f"IF EXISTS(SELECT * FROM sys.databases WHERE name='{sql_db_name}') DROP DATABASE {sql_db_name}")

    # Recreate the database
    server_cursor.execute(f"CREATE DATABASE {sql_db_name}")
    server_cursor.close()
    server_conn.close()

    # Now, connect to the newly created database
    conn = pyodbc.connect(database_connection_string)
    cursor = conn.cursor()

    # Ensure you're using the existing database
    cursor.execute(f"USE {sql_db_name}")

    # Create tables and populate them
    cursor.execute("""
    CREATE TABLE Customers (
        name VARCHAR(255),
        account_id INT PRIMARY KEY,
        loyalty_points INT,
    )
    """)

    for customer in customers:
        cursor.execute("INSERT INTO Customers VALUES (?, ?, ?)", 
                    (customer["name"], customer["account_id"], customer["loyalty_points"]))

    cursor.execute("""
    CREATE TABLE Products (
        id INT PRIMARY KEY,
        name VARCHAR(255),
        stock INT
    )
    """)

    for product in products:
        cursor.execute("INSERT INTO Products VALUES (?, ?, ?)", 
                    (product["id"], product["name"], product["stock"]))

    cursor.execute("""
    CREATE TABLE Orders (
        order_id INT PRIMARY KEY,
        product_id INT,
        days_to_delivery INT,
        account_id INT,
        FOREIGN KEY(product_id) REFERENCES Products(id),
        FOREIGN KEY(account_id) REFERENCES Customers(account_id)
    )
    """)

    for order in orders:
        cursor.execute("INSERT INTO Orders VALUES (?, ?, ?, ?)", 
                    (order["order_id"], order["product_id"], order["days_to_delivery"], order["account_id"]))

    conn.commit()

    #Verify database tables and columns
    def fetch_schema_info():
        cursor.execute("""
            SELECT t.TABLE_NAME, c.COLUMN_NAME, c.DATA_TYPE 
            FROM INFORMATION_SCHEMA.TABLES AS t
            JOIN INFORMATION_SCHEMA.COLUMNS AS c ON t.TABLE_NAME = c.TABLE_NAME
            WHERE t.TABLE_SCHEMA = 'dbo'  -- assuming you're using the default schema
            ORDER BY t.TABLE_NAME, c.ORDINAL_POSITION
        """)
        
        tables = {}
        for row in cursor.fetchall():
            table_name = row[0]
            column_name = row[1]
            data_type = row[2]
            
            if table_name not in tables:
                tables[table_name] = []
            
            tables[table_name].append(f"{column_name} ({data_type})")
        
        return tables

    schema_info = fetch_schema_info()

    # Print the schema info in a user-friendly format
    for table, columns in schema_info.items():
        print(f"Table: {table}")
        for col in columns:
            print(f"    {col}")
        print()

    # Close connections
    cursor.close()
    conn.close()